## day 1 Q2

1-2. 설치

3. API 를 통해 Ollama Llama3 모델에 프롬프트 입력하고 결과 받기
   - ollama serve 구동 필요
   - 아래 셀 실행

4. Model phi3로 변경해서 해보기

5. 사전에 작성한 2개의 역할 중 무엇이 사용자 입력 프롬프트에 대해 더 잘 답할 수 있을지 답변하는 프롬프트 작성하기 (질문에 대한 답변을 하는 것이 아니라 2개의 역할 중 어떤 역할이 잘 답할지를 답변하도록 하기)
첫번째 역할: math teacher
두번째 역할: history teacher
사용자 입력 예) what is the Newton’s law? How did World War I end?

6. 5에서 작성한 프롬프트와 역할을 이용해, 사용자 입력에 적합한 역할로써 사용자 입력에 답하는 프롬프트 및 코드 작성하기

7. 오늘 배운 프롬프트 엔지니어링을 활용해 5번의 각 역할이 더 대답을 잘할 수 있도록 프롬프트 개선하기

8. 1부터 n 까지의 자연 수 중 m 개의 수를 list 를 받아 정렬하는 문제를 LLM 이 잘 풀 수 있도록 다양한 프롬프트 엔지니어링을 시도, 평가해보고 시도한 프롬프트의 테스트 케이스들에 대한 성공률 및 순위를 작성


In [24]:
import re

def determine_persona(question):
    # Define regex patterns for math and history questions
    math_pattern = r'\b(\d+|\+|\-|\*|\/|=|calculate|multiply|divide|formula)\b'  # basic arithmetic pattern
    history_pattern = r'\b(who|when|where)\b.+\b(president|king|war|invention|discovery)\b'  # basic history pattern

    # Check for math question
    if re.search(math_pattern, question):
        return "You are a math teacher. Calculate formula step by step. "
    
    # Check for history question
    elif re.search(history_pattern, question, re.IGNORECASE):
        return "You are a history teacher. Share your knowledge or please search for relevant databases or internet. "

    # Default case for other types of questions
    else:
        return "You are a general knowledge expert. Provide a well-informed response. "

# Example usage
questions = [
    "what is 2 + 3?",
    "who was the first president of the USA?",
    "what is the capital of France?"
]

for question in questions:
    persona_header = determine_persona(question)
    print(f"Question: {question}")
    print(f"Persona Header: {persona_header}\n")

Question: what is 2 + 3?
Persona Header: You are a math teacher. Calculate formula step by step. 

Question: who was the first president of the USA?
Persona Header: You are a history teacher. Share your knowledge or please search for relevant databases or internet. 

Question: what is the capital of France?
Persona Header: You are a general knowledge expert. Provide a well-informed response. 



In [10]:
import os
import openai
import requests
import json

def question_on_api(question):
    # Define the API endpoint
    api_endpoint = "http://localhost:11434/api/chat"
    
    # Define the headers
    headers = {
        "Content-Type": "application/json"
    }
    
    prompt = determine_persona(question) + question
    
    # Define the payload
    payload = {
        "model": "llama3",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    
    # Make a POST request to the API
    response = requests.post(api_endpoint, headers=headers, json=payload)
    
    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Split the response into individual JSON objects
            raw_response = response.text
            json_objects = raw_response.splitlines()
            
            # Collect all content fields
            content_list = []
            for obj in json_objects:
                parsed_obj = json.loads(obj)
                content_list.append(parsed_obj['message']['content'])
            
            # Concatenate all content fields into a single sentence
            full_response = ''.join(content_list)
            print("Concatenated response:", full_response)
        except json.JSONDecodeError as e:
            # Print the raw response text if JSON decoding fails
            print("Failed to decode JSON response. Raw response:")
            print(response.text)
    else:
        print(f"Failed to generate text. Status code: {response.status_code}")
        print("Response:", response.text)

In [11]:
question_on_api("Who was the second president of USA?")

Concatenated response: As a history teacher, I'd be happy to share my knowledge with you!

The second President of the United States was John Adams.

John Adams served as the second President of the United States from 1797 to 1801. He was a Founding Father and one of the most important figures in American history. Prior to his presidency, Adams played a leading role in the drafting and signing of the Declaration of Independence in 1776.

Adams was a Federalist and served as Vice President under George Washington from 1789 to 1797. When Washington declined to run for re-election, Adams became the first vice president to assume the presidency.

During his presidency, Adams faced several challenges, including the XYZ Affair with France, which led to an undeclared naval war between the two countries. He also signed the Alien and Sedition Acts, which were controversial laws that restricted immigration and freedom of speech.

Adams was succeeded by Thomas Jefferson as president in 1801.

Sou

In [12]:
question_on_api("Can you calculate 3 * 2 / 6?")

Concatenated response: I'd be happy to walk you through the calculation step by step.

To calculate 3 * 2 / 6, we need to follow the order of operations (PEMDAS):

1. Multiply 3 and 2:
3 * 2 = 6

So far, our calculation looks like this:

6 / 6

2. Divide 6 by 6:
6 ÷ 6 = 1

And that's the final answer!

So, 3 * 2 / 6 = 1


In [13]:
question_on_api("I am an engineer with two kids. When I am tired, what can I do?")

Concatenated response: A fellow engineer! As someone who understands the demands of your profession and the importance of recharging, I'm happy to share some tips on how you can take care of yourself when feeling tired.

**Prioritize Self-Care**

As an engineer with a busy schedule, it's essential to prioritize self-care. When you're exhausted, your body and mind need a break. Here are some simple yet effective ways to recharge:

1. **Get enough sleep**: Aim for 7-9 hours of sleep each night. Establish a consistent sleep schedule, and create a relaxing bedtime routine to signal your brain that it's time to wind down.
2. **Take power naps**: If you're feeling tired during the day, try taking a 15-20 minute power nap. This can help refresh your mind and body. Just be sure to set an alarm so you don't oversleep!
3. **Engage in relaxation techniques**: Practice deep breathing exercises, progressive muscle relaxation, or meditation to calm your mind and reduce stress.
4. **Take breaks**: As

### 8. 

In [21]:
sort_prompt = """For example,
If the question was: Sort the following list of m natural numbers taken from 1 to n in ascending order:
Input: [8, 2, 5, 11, 6]
Steps:
1. Divide: Split the array into two halves.
2. Conquer: Recursively sort each half.
3. Combine: Merge the two sorted halves into a single sorted array.
Expected Output: [2, 5, 6, 8, 11].

Question: Please sort [10000, -2, 99, -6, 0]"""

- bad

In [23]:
question_on_api("Question: Please sort [10000, -2, 99, -6, 0]")

Concatenated response: A great question!

To sort this list of numbers, we'll use the basic concept of ordering numbers from smallest to largest. Here's the step-by-step calculation:

1. Start with the original list: `[10000, -2, 99, -6, 0]`
2. Compare the first two elements: `-2` and `0`. Since `-2` is smaller than `0`, we'll move `-2` to the front of the list.
	* New list: `[-2, 0, 10000, -6, 99]`
3. Now compare the next two elements: `0` and `10000`. Clearly, `0` is much smaller than `10000`, so we'll keep `-2` as the smallest element.
	* New list: `[-2, 0, 10000, -6, 99]`
4. Move on to the next pair of elements: `-6` and `99`. Since `-6` is smaller than `99`, we'll place `-6` after `0`.
	* New list: `[-2, 0, -6, 99, 10000]`

And that's it! The sorted list is:

`[-2, 0, -6, 99, 10000]`

Now, if you want to get fancy and use a more efficient sorting algorithm (like quicksort or mergesort), I'd be happy to help with that too!


- enhancement!

In [22]:
question_on_api(sort_prompt)

Concatenated response: To sort the given list of numbers in ascending order using the merge sort algorithm, I will follow these steps:

1. **Divide**: Split the array into two halves.

```
[10000, -2, 99, -6, 0]
```

Splitting at the middle index (since the length is odd), we get:

```
Left Half: [-2, 0, 99]
Right Half: [10000, -6]
```

1. **Conquer**: Recursively sort each half.

**Left Half**: Sort in ascending order.
```
[-2, 0, 99]
```

* Divide: Split the array into two halves.
	+ Left Half: [-2, 0]
	+ Right Half: [99]
* Conquer: Recursively sort each half.
	+ Left Half: [-2, 0] is already sorted. No further division or recursion needed.
	+ Right Half: [99] has only one element. It's already sorted. No further division or recursion needed.
* Combine: Merge the two sorted halves into a single sorted array.
	+ [-2, 0]

**Right Half**: Sort in ascending order.
```
[10000, -6]
```

* Divide: Split the array into two halves.
	+ Left Half: [-6]
	+ Right Half: [10000]
* Conquer: Recursiv